In [1]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
import torch.nn as nn
from tqdm import tqdm 
from torch.optim.lr_scheduler import ReduceLROnPlateau


/home/mmc/anaconda3/envs/CPR_YOLO/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2"

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'


train_df = pd.read_csv('../data/csv/train/drink_all.csv')
val_df = pd.read_csv('../data/csv/val/drink_all.csv')
num_classes = 1044

print(f'train shape: {train_df.shape}\nval shape: {val_df.shape}')

train shape: (119982, 18)
val shape: (15708, 18)


In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None):
        self.img_labels = annotations_file
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx]['path']
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels.iloc[idx]['class']
        if self.transform:
            image = self.transform(image)
        return image, label


# 데이터 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
])

# 사용자 정의 데이터셋 인스턴스 생성
train_dataset = CustomImageDataset(train_df,transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = CustomImageDataset(val_df, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)


In [4]:
class CustomImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CustomImageClassifier, self).__init__()
        self.base_model = timm.create_model('resnet50.a1_in1k', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        x = self.base_model(x)
        return x

In [5]:
# 모델 생성 및 사전 학습된 가중치 로드
model = CustomImageClassifier(num_classes)

# 장치 설정
model.to(device)

# 손실 함수 및 최적화 설정
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1, verbose=True)


/home/mmc/anaconda3/envs/CPR_YOLO/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [6]:
# 초기 최소 검증 손실값 설정
min_val_loss = float('inf')

# 학습 및 검증 과정
model.train()
for epoch in range(50):  # 10 에폭 동안 학습
    train_loss, val_loss = 0.0, 0.0
    
    # 학습 부분
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # 검증 부분
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    # 에폭별 손실 출력
    print(f'Epoch {epoch+1}, Train Loss: {train_loss / len(train_loader)}, Val Loss: {val_loss / len(val_loader)}')

    # 스케줄러 업데이트
    scheduler.step(val_loss / len(val_loader))

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    current_val_loss = val_loss / len(val_loader)
    if current_val_loss < min_val_loss:
        min_val_loss = current_val_loss
        torch.save(model.state_dict(), f'./drink_pt/model_epoch_{epoch+1}_loss_{current_val_loss:.4f}.pt')
        print(f"Model saved: Epoch {epoch+1} with Val Loss: {current_val_loss:.4f}")

100%|██████████| 246/246 [00:50<00:00,  4.91it/s]


Epoch 1, Train Loss: 0.5613193798017998, Val Loss: 0.10863177269712333
Model saved: Epoch 1 with Val Loss: 0.1086


100%|██████████| 246/246 [00:45<00:00,  5.44it/s]


Epoch 2, Train Loss: 6.948643260025978, Val Loss: 6.952855852561268


100%|██████████| 246/246 [00:49<00:00,  4.98it/s]


...

Epoch 48, Train Loss: 6.947090728759766, Val Loss: 6.956974583912671


100%|██████████| 246/246 [00:29<00:00,  8.28it/s]


Epoch 49, Train Loss: 6.947086618550618, Val Loss: 6.95701861575367


100%|██████████| 246/246 [00:29<00:00,  8.26it/s]

Epoch 50, Train Loss: 6.947090070851644, Val Loss: 6.957002740565354


In [7]:
import matplotlib.pyplot as plt

# plt.imshow(images[2].to('cpu').permute(1,2,0))
print(outputs[0])
print(labels.to('cpu'))

tensor([-0.3605, -0.0821, -0.0666,  ..., -0.1296, -0.0384, -0.0499],
       device='cuda:0')
tensor([ 65, 473, 342, 613, 546, 442, 782, 127, 553, 571, 785, 128, 360, 850,
        453, 588, 225, 719,  75, 625, 983, 330, 371, 852,  24, 964,  55, 156])
